### Import Libraries

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import os
os.environ['KERAS_BACKEND'] = "tensorflow"
#TensorFlow: Channels last order.
#Theano: Channels first order.
#CNTK: Channels last order.

In [ ]:
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import imageio as im
from keras import models
from keras.datasets import fashion_mnist
from keras.models import load_model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras import backend as K
#K.set_image_dim_ordering('tf') 
K.image_data_format() == 'channels_last'
from keras.utils import np_utils

### Load and Visualize Data

In [ ]:
# View 6 images of FashionMNIST in a gray scale
def visualize_fashion_mnist(X_train):
    plt.subplot(321)
    plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
    plt.subplot(322)
    plt.imshow(X_train[1], cmap=plt.get_cmap('gray'))
    plt.subplot(323)
    plt.imshow(X_train[2], cmap=plt.get_cmap('gray'))
    plt.subplot(324)
    plt.imshow(X_train[3], cmap=plt.get_cmap('gray'))
    plt.subplot(325)
    plt.imshow(X_train[4], cmap=plt.get_cmap('gray'))
    plt.subplot(326)
    plt.imshow(X_train[5], cmap=plt.get_cmap('gray'))
    plt.show()
    
def data_preparation():
    
    ((X_train, y_train), (X_test, y_test)) = fashion_mnist.load_data()
    
    visualize_fashion_mnist(X_train)
    
    # Transform to the format [instances][width][height][pixels]
    # (60000, 28, 28)
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
    # (60000, 28, 28, 1)

    # Normalize the pixel values from 0-255 to 0-1
    X_train = X_train / 255
    X_test = X_test / 255

    # Transform the label which is an integer into binary categories
    # The value becomes that corresponding to the position, class 5 becomes the list [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]
    
    return X_train, X_test, y_train, y_test, num_classes

X_train, X_test, y_train, y_test, num_classes = data_preparation()

### Model Execution

In [ ]:
def create_model_cnn_simple(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def create_model_cnn_plus(num_classes):
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def create_model_cnn_plus_plus(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape = (28, 28, 1), activation = 'relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 
    model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 
    model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation = 'softmax'))
    return model

def create_model_cnn_lenet5(num_classes):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPool2D(strides=2))
    model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
    model.add(MaxPool2D(strides=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

model = create_model_cnn_plus(num_classes)

In [ ]:
model.summary()

In [ ]:
# Compile model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
checkpointer = ModelCheckpoint(filepath="best_weights_fashion_mnist.hdf5", monitor = 'val_accuracy', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=200, verbose=1, callbacks=[checkpointer])

In [ ]:
model.load_weights('best_weights_fashion_mnist.hdf5')

In [ ]:
# To save:
model.save('fashion_mnist_cnn.h5')
#model.save('fashion_mnist_cnn.h5')

# To use:
#model=load_model('fashion_mnist_cnn.h5')

In [ ]:
# Utils to visualize the history of the learning
def print_history_accuracy(history):
    #print(history.history.keys())
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def print_history_loss(history):
    #print(history.history.keys())
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

print_history_accuracy(history)
print_history_loss(history)

In [ ]:
# View image, label and corresponding prediction
def visualize_prediction(x_test, y_test, img_to_show):
    print("X_test:",x_test.shape)
    print("y_test:",y_test.shape)
    print("Image:",x_test[img_to_show].shape)
    plt.imshow(x_test[img_to_show,:,:,0], cmap='gray')
    plt.show()
    print("Label:",y_test[img_to_show])
    print("Before increasing another dimension:",x_test[img_to_show].shape)
    image_tensor = np.expand_dims(x_test[img_to_show], axis=0)
    print("After increasing another dimension:",image_tensor.shape)
    print("Prediction:",model.predict(image_tensor))
    classes = model.predict_classes(image_tensor)
    print('Predicted Class:',classes)
    return image_tensor


image_tensor = visualize_prediction(X_test, y_test, 3)

### Featuremaps Visualization

In [ ]:
# Let's get the outputs of the first 5 (plus) layers of the network or 12 for plus_plus
output_layers = []
for layer in model.layers[:5]:
    print(layer.output.shape)
    output_layers.append(layer.output)

# And then we create a model that returns these outputs given the model inputs
activation_model = models.Model(inputs=model.input, outputs=output_layers)

In [ ]:
activations = activation_model.predict(image_tensor) 
# Returns a list of two Numpy arrays: one array per activation layer

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
plt.imshow(first_layer_activation[0, :, :, 9], cmap='viridis')
plt.show()

In [ ]:
layers_names = []
for layer in model.layers[:5]:
    layers_names.append(layer.name)
    
images_per_line = int(first_layer_activation.shape[3] / 2)

for layer_name, activation_layer in zip(layers_names, activations):
    n_features = activation_layer.shape[-1]
    size = activation_layer.shape[1]
    n_lines = -(-n_features // images_per_line)
    print("layer_name:",layer_name)
    print("n_features:",n_features)
    print("size:",size)
    print("n_lines:",n_lines)
    display_grid = np.zeros((size * n_lines, images_per_line * size))
    for col in range(n_lines):
        for lin in range(images_per_line):
            image = activation_layer[0,:,:,col * images_per_line + lin]
            image -= image.mean()
            image /= image.std()
            image *= 64
            image += 128
            image = np.clip(image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,lin * size : (lin + 1) * size] = image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')